## Extract the text from pdf and save as Excel file

In [ ]:
from img2table.document import PDF
import numpy as np

# PaddleOCR
from img2table.ocr import PaddleOCR

paddle_ocr = PaddleOCR(lang="en", kw={"use_dilation": True})

pdf = PDF(src=r"D:\Python\ajit\text_extract\try2\try6\new_pdf\16933407.pdf")
# Table extraction
extracted_tables = pdf.extract_tables(ocr=paddle_ocr,
                                      implicit_rows=True,
                                      borderless_tables=True)

pdf.to_xlsx(r'D:\Python\ajit\text_extract\try2\try6\op\16933407.xlsx',ocr=paddle_ocr,borderless_tables=True, implicit_rows=True)


In [ ]:
# pdf_table extraction
import os
from img2table.document import PDF
import numpy as np
from img2table.ocr import PaddleOCR
paddle_ocr = PaddleOCR(lang="en", kw={"use_dilation": True})


file_name = r"D:\Python\ajit\text_extract\try2\try6\new_pdf"
op_path1 = r"D:\Python\ajit\text_extract\try2\try6\op\pdf_to_table"

for i in os.listdir(file_name):
    v = (i.split("."))
    pdf_path = os.path.join(file_name,i)
    pdf = PDF(src=pdf_path)
    extracted_tables =pdf.extract_tables(ocr=paddle_ocr,implicit_rows=True,borderless_tables=True)
    op_path = os.path.join(op_path1,i)
    a = op_path.replace(".pdf",".xlsx")
    pdf.to_xlsx(a,ocr=paddle_ocr,borderless_tables=True, implicit_rows=True)
    print(a)
    
    

## Extract the key information from excel and save output in csv

In [1]:
## Main def function

import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

def process_e1(a):
    e1 = []
    for sublist in a:
        for item in sublist:
            c = str(item).replace("(", "").replace(")", "").replace(",", "").split("\n")
            e1.append(c)
    
    return e1


# Function to remove the item before the string starting with '*** Exit Load Deducted' and store in separate list
def process_exit_load(data):
    removed_items = []
    for sublist in data:
        indices_to_remove = []
        for i in range(1, len(sublist)):
            if sublist[i].startswith('*** Exit Load Deducted'):
                indices_to_remove.append(i - 1)
        # Remove items in reverse order to avoid index shifting issues
        for index in sorted(indices_to_remove, reverse=True):
            removed_items.append(sublist[index])
            del sublist[index]
    return data


def cleaned_data(removed_items):
    b1 =[]
    for i in removed_items:
        for j in i:
            if j[:1].isdigit():
                b1.append(j)
    return b1

def removed_lien(b):
    re = []
    n = []
    for i in a:
        for j in i:
            b = str(j)
            if b.startswith("***Lien") or b.startswith("***Invalid") or b.startswith("***Multiple"):
                re.append(b)
            else:
                n.append([b])
    return n
            

def transaction_type(removed_items):
    t = []
    for i in removed_items:
        for j in i:
            if j.lower().startswith("redemption") :
                t.append(j)
            if j.lower().startswith("purchase"):
                t.append(j)
    return t


def company_name(a):
    com = []
    for i in a[0:2]:
        for j in i:
            v = str(j).lower().split("\n")
            for k in v:
                if k.endswith("ltd") or k.endswith("pvt ltd") or k.endswith("limited"):
                    com.append(k)
    return com


def create_groups(data, start_position):
    groups = []
    length = len(data)
    
    # Iterate through the list in steps of 5 starting from the start_position
    for i in range(start_position, length, 6):
        group = data[i:i + 6]  # Get the next 5 items
        groups.append(group)   # Append the group to the groups list
    
    return groups

def groups_to_dict(groups):
    data_dicts = []
    for group in groups:
        data_dict = {
            'Balance_Unit': group[0],  # Replace 'field1', 'field2', etc., with actual field names
            'Amount(INR)': group[1],
            'NAV(INR)': group[2],
            'Price(INR)': group[3],
            'No_of_unit': group[4]
            # Add as many fields as necessary based on the structure of `group`
        }
        data_dicts.append(data_dict)
    return data_dicts

def filter_digit_starting_items(e1):
   z = []
   for i in e1:
      s = "Levied"
      if not any(s in item for item in i):
         z.append(i)
   return z

def insi_num(a):
    v = []
    for i in a:
        for j in i:
            c1 = str(j)
            c = (c1.split("/"))
            for k in c:
                if k.lower().startswith("inf") or k.lower().startswith(" inf"):
                    v.append(k)
    return v

predefined_string = "Amount, Transaction Type, NAV, Price, NO_of_Units, Balance_Units"


In [2]:
def process_pdf_text(pdf_path):
    """
    Function to extract text from all pages of a given PDF.
    """
    reader = PdfReader(pdf_path)
    num_pages = len(reader.pages)
    combined_text = ""
    
    for i in range(num_pages):
        page = reader.pages[i]
        text = page.extract_text()
        if text:
            combined_text += text
    
    return combined_text


def date_removed(b2):
    date_pattern = r"\b\d{2}/\d{2}/\d{4}\b"
    filtered_list = [item for item in b2 if not re.match(date_pattern, item)]
    return filtered_list


def removed_change(list_of_lists):
    # Use list comprehension to filter out sublists containing the word "Levied"
    filtered_list = [sublist for sublist in list_of_lists if not any("***Change of" in str(item) for item in sublist)]
    return filtered_list


def removed_invalid(list_of_lists):
    # Use list comprehension to filter out sublists containing the word "Levied"
    filtered_list = [sublist for sublist in list_of_lists if not any("***Invalid" in str(item) for item in sublist)]
    return filtered_list


def com_name(c2):
    com =[]
    for i in c2:
        j = str(i).split("\n")
        com.append(j[0])
    return com


import re

def extract_values_from_date_regex(data_list):
    data_str = ' '.join(data_list)
    pattern = r'(\b\d{2}-(?:[A-Za-z]{3})-\d{4}\b|\b\d{2}/\d{2}/\d{4}\b)(?:\s+(\S+)){5}'
    
    matches = re.finditer(pattern, data_str)
    extracted_values = []
    for match in matches:
        values = match.group() 
        values_list = values.split()  
        extracted_values.append(values_list)
    
    return extracted_values



def removed_levied(list_of_lists):
    # Use list comprehension to filter out sublists containing the word "Levied"
    filtered_list = [sublist for sublist in list_of_lists if not any("Levied" in str(item) for item in sublist)]
    return filtered_list


def process_exit_load1(list_of_lists):
    # Use list comprehension to filter out sublists containing the word "Levied"
    filtered_list = [sublist for sublist in list_of_lists if not any("Exit Load" in str(item) for item in sublist)]
    return filtered_list

def removed_address(list_of_lists):
    # Use list comprehension to filter out sublists containing the word "Levied"
    filtered_list = [sublist for sublist in list_of_lists if not any("***Address" in str(item) for item in sublist)]
    return filtered_list

def process_e1(a):
    e1 = []
    for sublist in a:
        for item in sublist:
            c = str(item).replace("(", "").replace(")", "").replace(",", "").split("\n")
            e1.append(c)
    
    return e1


def transaction_type(removed_items):
    t = []
    for i in removed_items:
        for j in i:
            if j.lower().startswith("redemption") :
                t.append(j)
            if j.lower().startswith("purchase"):
                t.append(j)
    return t


def balance_unit_pop(e):
    i1 = []
    for i in e:
        if len(i) ==2:
            i1.append(i.pop(0))
    return e


def process_exit_load(data):
    removed_items = []
    for sublist in data:
        indices_to_remove = []
        for i in range(1, len(sublist)):
            if sublist[i].startswith('*** Exit Load Deducted') :
                indices_to_remove.append(i - 1)
        # Remove items in reverse order to avoid index shifting issues
        for index in sorted(indices_to_remove, reverse=True):
            removed_items.append(sublist[index])
            del sublist[index]
    return data


def cleaned_data(removed_items):
    b1 =[]
    for i in removed_items:
        for j in i:
            if j[:1].isdigit():
                b1.append(j)
    return b1


def insi_number_extract(pdf_path):
    reader = PdfReader(pdf_path)
    isin_pattern = r"INF"
    extracted_data = []
    num_pages = len(reader.pages)
    for i in range(num_pages):
        page = reader.pages[i]
        text = page.extract_text()
        tes = text.replace(',', '').replace("(", "").replace(')', '').replace("NAV", "")
        indices = [m.start(0) for m in re.finditer(isin_pattern, tes)]
        
        for i in indices:
            vv = tes[i:]
            if "RedemptionElectronic" in vv or "Purchase DT" in vv or "RedemptionRTGS" in vv:
                vb = tes[i:i+13]
                vvv = vb.replace('\n', "")
                extracted_data.append(vvv)
                
    return extracted_data


import shutil

def process_pdf_text(pdf_path):
    """
    Function to extract text from all pages of a given PDF.
    """
    reader = PdfReader(pdf_path)
    num_pages = len(reader.pages)
    combined_text = ""
    
    for i in range(num_pages):
        page = reader.pages[i]
        text = page.extract_text()
        if text:
            combined_text += text
    
    return combined_text

def extract_info_from_text(tes):
    """
    Function to extract relevant ISIN-related information from text.
    """
    tes = tes.replace(',', '').replace("(", "").replace(')', '').replace("NAV", "")
    indices = [m.start(0) for m in re.finditer(isin_pattern, tes)]

    extracted_data = []
    for i in indices:
        vv = tes[i:i+300]
        if "Redemption" in vv or "Purchase" in vv:
            vb = tes[i:i+13]
            vvv = vb.replace('\n', "")
            extracted_data.append(vvv)
    
    return extracted_data



def extract_insi_and_folio(pdf_path):
    reader = PdfReader(pdf_path)
    folio_pattern = r'Folio Number\s*:\s*(\d+(?:\s*/\s*\d+)?)'
    isin_pattern = r"INF"
    extracted_folio = []
    extracted_insi = []
    folio = []
    num_pages = len(reader.pages)
    for i in range(num_pages):
        page = reader.pages[i]
        text = page.extract_text()
        tes = text.replace(',', '').replace("(", "").replace(')', '').replace("NAV", "")
        indices = [m.start(0) for m in re.finditer(isin_pattern, tes)]
        indices1 = [m.start(0) for m in re.finditer(folio_pattern, tes)]
        for i in indices:
            vv = tes[i:]
            if "RedemptionElectronic" in vv or "Purchase DT" in vv or "RedemptionRTGS" in vv or "Purchase" in vv:
                vb = tes[i:i+13]
                vvv = vb.replace('\n', "")
                extracted_insi.append(vvv)
            
        for i in indices1:
            vv = tes[i:]
            if "RedemptionElectronic" in vv or "Purchase DT" in vv or "RedemptionRTGS" in vv or "Purchase" in vv:
                vb = tes[i:i+23]
                vvv = vb.replace('\n', "")
                extracted_folio.append(vvv)

    for i in extracted_folio:
        b = (i.split(':'))
        folio.append(b[1])
        
    return extracted_insi,folio
  

def extract_folio_icici(pdf_path):
    reader = PdfReader(pdf_path)
    folio_pattern = r'Folio\s+No\s*\.\s*(\d+\s*/\s*\d+)'
    extracted_folio = []
    folio = []
    num_pages = len(reader.pages)
    for i in range(num_pages):
        page = reader.pages[i]
        text = page.extract_text()
        tes = text.replace(',', '').replace("(", "").replace(')', '').replace("NAV", "")
        indices1 = [m.start(0) for m in re.finditer(folio_pattern, tes)]
                
        for i in indices1:
            vv = tes[i:]
            if "RedemptionElectronic" in vv or "Purchase DT" in vv or "RedemptionRTGS" in vv or "Purchase" in vv or "Redemption" in vv:
                vb = tes[i:i+24]
                vvv = vb.replace('\n', "")
                extracted_folio.append(vvv)

    for i in extracted_folio:
        b = (i.split('.'))
        folio.append(b[1])

    return folio


In [3]:
### Compile all three code in single script

import os
import re
import pandas as pd
import shutil
from PyPDF2 import PdfReader

# Main directories for storing all files
base_dir = r"D:\Python\ajit\combined_all_code"
pdf_dir = os.path.join(base_dir, "pdfs")
excel_dir = os.path.join(base_dir, "excels")
output_dir = os.path.join(base_dir, "op1")

# Ensure base directories exist
os.makedirs(pdf_dir, exist_ok=True)
os.makedirs(excel_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

def extract_text_from_pdf(pdf_file_path):
    # print(pdf_file_path)
    reader = PdfReader(pdf_file_path)
    page = reader.pages[0]
    text = page.extract_text()
    # Dummy function. Replace with actual PDF text extraction code.
    return text


# Function to check if the PDF file matches the condition (SBI, Birla, ICICI, etc.)
def check_file_type(pdf_file_path):
    # Replace this with actual PDF text extraction logic
    pdf_text = extract_text_from_pdf(pdf_file_path)
    
    # Define patterns for different categories (SBI, Birla, ICICI)
    patterns = {
        'sbi': r"\b(sbimf|sbi premiers liquid fund|sbi mutual fund)\b",
        'birla': r"\b(aditya birla sun life|aditya birla sun life liquid fund|absl liquid fund)",
        'icici': r"\b(icici|icici prudential)\b"
    }
    
    # Check which pattern matches the PDF text
    for key, pattern in patterns.items():
        if re.search(pattern, pdf_text, re.IGNORECASE):
            return key
    return None


for pdf_file in os.listdir(pdf_dir):
    pdf_path = os.path.join(pdf_dir, pdf_file)
    print(f"Processing pdf: {pdf_path}")
    file_type = check_file_type(pdf_path)

    if file_type =='sbi':
        print('I AM IN SBI')
        # Create specific directories for the file type (e.g., sbi, birla, icici)
        specific_pdf_dir = os.path.join(base_dir,'op1',file_type, "pdf")
        specific_excel_dir = os.path.join(base_dir, 'op1',file_type, "excel")
        specific_output_dir = os.path.join(base_dir, 'op1',file_type, "csv")

        # Ensure directories exist
        os.makedirs(specific_pdf_dir, exist_ok=True)
        os.makedirs(specific_excel_dir, exist_ok=True)
        os.makedirs(specific_output_dir, exist_ok=True)
        isin_pattern = r"INF"
        all_groups = []
        com_nam = []
        folio = []
        len_u = []
        inf = []
        folio_pattern = r'Folio Number\s*:\s*(\d+(?:\s*/\s*\d+)?)'

        excel_filename = pdf_file.replace('.pdf', '.xlsx')  
        excel_path = os.path.join(excel_dir, excel_filename)
        print(excel_path,'0000000000000000')
     
        if excel_path.endswith(".xlsx"):
            xls = pd.ExcelFile(excel_path)
            table_names = xls.sheet_names
            df1 = pd.read_excel(xls, sheet_name="Page 1 - Table 1", header=None)
            a1 = df1.values.tolist()
            c2 = []
            c = a1[0:2][1][0]
            c2.append(c)
            com_na = com_name(c2)
            print(com_na)
            com_nam.extend(com_na)
            all_tran = []
            for sheet in table_names:
                df = pd.read_excel(xls, sheet_name=sheet, header=None)
                a = df.values.tolist()
                a1 = removed_invalid(a)
                a2 = removed_address(a1)
                cl = removed_levied(a2)
                ch = removed_change(cl)
                re1 = process_exit_load1(ch)
                e = process_e1(re1)
                u = transaction_type(e)
                b = balance_unit_pop(e)
                removed_items = process_exit_load(b)
                b2 = cleaned_data(removed_items)
                v = date_removed(b2)
                all_tran.extend(u)
                all_tran1 = [item for item in all_tran if item != 'Redemption']
                if len(u) !=0:
                    groups = extract_values_from_date_regex(v)
                    le = len(groups)
                    len_u.append(le)
                    all_groups.extend(groups)
                        
            df = pd.DataFrame(all_groups)
            if df.shape[1] == 0:
                com_nam.clear()

            else:
                if df.shape[1] > 2:
                    corresponding_pdf_file = excel_filename.replace('.xlsx', '.pdf')
                    pdf_path = os.path.join(pdf_dir, corresponding_pdf_file)
                    if os.path.exists(pdf_path):
                        extracted_info,folio_info = extract_insi_and_folio(pdf_path)
                        inf.extend(extracted_info)

                    kk = [item for item in len_u if item != 0]
                    result = [s for s, n in zip(inf, kk) for _ in range(n)]
                    kk1 = [item for item in len_u if item != 0]
                    result1 = [s for s, n in zip(folio_info, kk) for _ in range(n)]

                    df.columns = ['Value Date','Amount in INR', 'NAV in INR', 'Price in INR', 'Number of Units', 'Balance Units']
                    df['Company Name'] = pd.Series([com_nam[0]] * len(df), index=df.index) 
                    df['Transaction Type'] = pd.Series(all_tran1)
                    df['Folio Number'] = pd.Series(result1)
                    df['ISIN Number'] = pd.Series(result)
                    df['Transaction Type'] = df['Transaction Type'].apply(lambda x: 'Purchase' if str(x).lower().startswith('pu') else 'Redemption')

                    new_order = ['Folio Number','Company Name','ISIN Number','Value Date','Transaction Type','Amount in INR','NAV in INR','Price in INR','Number of Units','Balance Units']
                    df1 = df[new_order]
                    print(df1)
        
                    # Save the DataFrame to CSV
                    csv_filename = f"{os.path.splitext(excel_filename)[0]}.csv"
                    csv_path = os.path.join(specific_output_dir,csv_filename)
                    df1.to_csv(csv_path, index=False)
                    print(f"Data saved to {csv_path}")

                    all_groups.clear()
                    com_nam.clear()
                    all_tran1.clear()
                    inf.clear()
                    len_u.clear()

            # new_pdf_path = os.path.join(specific_pdf_dir, pdf_file)
            # shutil.move(pdf_path, new_pdf_path)
            # print(f"Moved PDF to: {new_pdf_path}")

            xls.close()
            # new_excel_path = os.path.join(specific_excel_dir, excel_filename)
            # shutil.move(excel_path, new_excel_path)
            # print(f"Moved Excel to: {new_excel_path}")

    if file_type =='birla':
        print('I AM IN BIRLA')
        # Create specific directories for the file type (e.g., sbi, birla, icici)
        specific_pdf_dir = os.path.join(base_dir,'op1',file_type, "pdf")
        specific_excel_dir = os.path.join(base_dir, 'op1',file_type, "excel")
        specific_output_dir = os.path.join(base_dir, 'op1',file_type, "csv")

        # Ensure directories exist
        os.makedirs(specific_pdf_dir, exist_ok=True)
        os.makedirs(specific_excel_dir, exist_ok=True)
        os.makedirs(specific_output_dir, exist_ok=True)
        
        folio_numbers_pattern = r'Folio Number\s*:\s*(\d+(?:\s*/\s*\d+)?)'
        isin_pattern = r"INF209"

        # Lists to store extracted data
        len_u = []
        t = []
        inf = []
        folio_list = []
        inf_n = []
        co1= []
        excel_filename = pdf_file.replace('.pdf', '.xlsx')  # Replace .pdf with .xlsx   
        excel_path = os.path.join(excel_dir, excel_filename)
        print(excel_path,'0000000000000000')

        if excel_path.endswith(".xlsx"):
            xls = pd.ExcelFile(excel_path)
            table_names = xls.sheet_names
            df1 = pd.read_excel(xls, sheet_name="Page 1 - Table 1", header=None)
            a1 = df1.values.tolist()
            c = company_name(a1)
            c2 = []
            if len(c) == 0:
                c2.append(a1[1])
            else:
                c2.append(c)
            
            for i in c2:
                for j in i:
                    co1.append(j)

            all_groups = []
            all_tran = []
            for sheet in table_names:
                df = pd.read_excel(xls, sheet_name=sheet, header=None)
                a = df.values.tolist()
                for i in a[0:2]:
                    s = str(i)
                    vector1 = text_to_vector(predefined_string)  
                    vector2 = text_to_vector(s)
                    cosine = get_cosine(vector1, vector2)
                    if cosine > 0.48:
                        df = pd.read_excel(xls, sheet_name=sheet, header=None)
                        cl = removed_levied(a)
                        re1 = process_exit_load1(cl)
                        e = process_e1(re1)
                        u = transaction_type(e) 
                        le = len(u)
                        len_u.append(le)
                        b = balance_unit_pop(e)
                        removed_items = process_exit_load(b)
                        b2 = cleaned_data(removed_items)
                        m = e[0][0]
                
                        if m.startswith("Value"):
                            groups = extract_values_from_date_regex(b2)
                        else:
                            groups = extract_values_from_date_regex(b2)

                        all_groups.extend(groups)
                        all_tran.extend(u)

            corresponding_pdf_file = excel_filename.replace('.xlsx', '.pdf')      
            pdf_path = os.path.join(pdf_dir, corresponding_pdf_file)

            if os.path.exists(pdf_path):
                print(f"Processing PDF: {pdf_path}")
                text = process_pdf_text(pdf_path)
                folio_numbers = re.findall(folio_numbers_pattern, text)
                folio_list.extend(folio_numbers)
                extracted_info = extract_info_from_text(text)
                inf.extend(extracted_info)

            kk = [item for item in len_u if item != 0]
            result = [s for s, n in zip(inf, kk) for _ in range(n)]
        
            # Create DataFrame using all_groups
            df = pd.DataFrame(all_groups)
            df.columns = ['Value Date','Amount in INR', 'NAV in INR', 'Price in INR', 'Number of Units', 'Balance Units']
            df['Company Name'] = pd.Series([co1[0]] * len(df), index=df.index) 
            df['Transaction Type'] = pd.Series(all_tran)
            if folio_list:
                df['Folio Number'] = pd.Series([folio_list[0]] * len(df), index=df.index)
            df['ISIN Number'] = pd.Series(result)
            df['Transaction Type'] = df['Transaction Type'].apply(lambda x: 'Purchase' if str(x).lower().startswith('pu') else 'Redemption')

            new_order = ['Folio Number','Company Name','ISIN Number','Value Date','Transaction Type','Amount in INR','NAV in INR','Price in INR','Number of Units','Balance Units']
            df1 = df[new_order]
            print(df1)
        
            # Save the DataFrame to CSV
            csv_filename = f"{os.path.splitext(excel_filename)[0]}.csv"
            csv_path = os.path.join(specific_output_dir,csv_filename)
            df1.to_csv(csv_path, index=False)
            print(f"Data saved to {csv_path}")

            # Clean up lists for next iteration
            inf.clear()
            folio_list.clear()
            all_groups.clear()
            all_tran.clear()
            co1.clear()
            len_u.clear()

            # # moved pdf
            # new_pdf_path = os.path.join(specific_pdf_dir, pdf_file)
            # shutil.move(pdf_path, new_pdf_path)
            # print(f"Moved PDF to: {new_pdf_path}")

            # # moved excel
            xls.close()
            # new_excel_path = os.path.join(specific_excel_dir, excel_filename)
            # shutil.move(excel_path, new_excel_path)
            # print(f"Moved Excel to: {new_excel_path}")

    if file_type =='icici':
        print('I AM IN ICICI')
        # Create specific directories for the file type (e.g., sbi, birla, icici)
        specific_pdf_dir = os.path.join(base_dir,'op1',file_type, "pdf")
        specific_excel_dir = os.path.join(base_dir, 'op1',file_type, "excel")
        specific_output_dir = os.path.join(base_dir, 'op1',file_type, "csv")

        # Ensure directories exist
        os.makedirs(specific_pdf_dir, exist_ok=True)
        os.makedirs(specific_excel_dir, exist_ok=True)
        os.makedirs(specific_output_dir, exist_ok=True)
        co=[]
        ins = []
        folio = []
        patterns = "pageFolio No.\s*([\d\s/]+)" 
        patterns1 = r'(?:page|\d+)Folio\s*No\.\s*(\d{8,10}\s*/\s*\d{1,3})'
        excel_filename = pdf_file.replace('.pdf', '.xlsx')  # Replace .pdf with .xlsx   
        excel_path = os.path.join(excel_dir, excel_filename)
        print(excel_path)

        if excel_path.endswith(".xlsx"):
            xls = pd.ExcelFile(excel_path)
            table_names = xls.sheet_names
            df1 = pd.read_excel(xls, sheet_name="Page 1 - Table 1", header=None)
            a1 = df1.values.tolist()
            c1 = company_name(a1)
            for i in c1:
                co.append(i)              
            all_groups = []
            all_tran = []
            all_isni = []
            insi = []
            ins1=[]
            for j in table_names:
                df = pd.read_excel(xls, sheet_name=j, header=None)
                a = df.values.tolist()
                for l in a[0:2]:
                    s = str(l)
                    vector1 = text_to_vector(predefined_string)
                    vector2 = text_to_vector(s)
                    cosine = get_cosine(vector1, vector2)
                    if cosine >= 0.07:
                        q = removed_lien(a)
                        v = process_e1(q)
                        z1 = filter_digit_starting_items(v)
                        removed_items = process_exit_load(z1)
                        b2 = cleaned_data(removed_items)
                        u = transaction_type(removed_items)
                        all_tran.extend(u)
                        ins = insi_num(a)
                        if len(u) >= 1:
                            insi.append(ins)
                        for i in insi:
                            for j in i:
                                ins1.append(j)

                        empty_list = [x for x in ins if x != []]
                        if len(u) < len(empty_list):
                            empty_list.pop(0)

                        m = v[0][0]
                        if m[:1].isdigit():
                            start_position = 2
                            groups = create_groups(b2, start_position)
                        else:
                            start_position = 0
                            groups = create_groups(b2, start_position)

                        all_groups.extend(groups)
                        all_isni.extend(empty_list * len(groups))

                df = pd.DataFrame(all_groups)
            
            # Check if DataFrame has more than 2 columns
            if df.shape[1] > 2:
                corresponding_pdf_file = excel_filename.replace('.xlsx', '.pdf')      
                pdf_path = os.path.join(pdf_dir, corresponding_pdf_file)
                print(pdf_path)
                if os.path.exists(pdf_path):
                    folio_num = extract_folio_icici(pdf_path)
                    folio.extend(folio_num)
                
                    # Create final DataFrame
                    df.columns = ['Amount in INR', 'NAV in INR', 'Price in INR', 'Number of Units', 'Balance Units','Value Date',]
                    cols = df.columns.tolist()
                    cols = cols[-1:] + cols[:-1]
                    df = df[cols]
                    df['Company Name'] = pd.Series([co[0]] * len(df), index=df.index)
                    df['Transaction Type'] = pd.Series(all_tran)
                    df['Folio Number'] = pd.Series([folio[0]] * len(df), index=df.index)
                    df['ISIN Number'] = pd.Series(all_isni)
                    co.pop(0)
                    folio.pop(0)
                    df['Transaction Type'] = df['Transaction Type'].apply(lambda x: 'Purchase' if str(x).lower().startswith('pu') else 'Redemption')
                    new_order = ['Folio Number','Company Name','ISIN Number','Value Date','Transaction Type','Amount in INR','NAV in INR','Price in INR','Number of Units','Balance Units']
                    df1 = df[new_order]
                    
                    df2 = df1.dropna() 
                    print(df2)

                    # save csv
                    csv_filename = f"{os.path.splitext(excel_filename)[0]}.csv"
                    csv_path = os.path.join(specific_output_dir,csv_filename)
                    df2.to_csv(csv_path, index=False)
                    print(f"Data saved to {csv_path}")
                    
                    # Move PDF file to a new folder based on Excel file path
                    # new_pdf_path = os.path.join(specific_pdf_dir, pdf_file)
                    # shutil.move(pdf_path, new_pdf_path)
                    # print(f"Moved PDF to: {new_pdf_path}")

                    xls.close()
                    # new_excel_path = os.path.join(specific_excel_dir, excel_filename)
                    # shutil.move(excel_path, new_excel_path)
                    # print(f"Moved Excel to: {new_excel_path}")
                        
                        

Processing pdf: D:\Python\ajit\combined_all_code\pdfs\1014759232 Adani Power Ltd.pdf
I AM IN BIRLA
D:\Python\ajit\combined_all_code\excels\1014759232 Adani Power Ltd.xlsx 0000000000000000
Processing PDF: D:\Python\ajit\combined_all_code\pdfs\1014759232 Adani Power Ltd.pdf
  Folio Number      Company Name    ISIN Number  Value Date Transaction Type  \
0   1014759232  Adani Power Ltd.  INF209K01VA3   13/08/2024       Redemption   
1   1014759232  Adani Power Ltd.   INF209KB1ZH2  08/08/2024       Redemption   
2   1014759232  Adani Power Ltd.   INF209KB1ZH2  08/08/2024       Redemption   
3   1014759232  Adani Power Ltd.   INF209KB1ZH2  11/08/2024       Redemption   
4   1014759232  Adani Power Ltd.   INF209KB1ZH2  11/08/2024         Purchase   

    Amount in INR NAV in INR Price in INR Number of Units Balance Units  
0    400000000.00   400.2464     400.2464     3370415.401    999384.379  
1    340000000.00  1325.3111    1325.3111      256543.539   5372622.863  
2    500000000.00  1325.

In [4]:
## main code for final_csv
import os
import pandas as pd
from datetime import date
today = date.today()
date1 = str(today)

main_folder = r"D:\Python\ajit\combined_all_code\op1"  
csv_data = []

for subfolder in ["icici", "sbi", "birla"]:
    csv_folder = os.path.join(main_folder, subfolder, "csv")
    des_path = os.path.join(main_folder, subfolder, 'move_csv')
    os.makedirs(des_path, exist_ok=True)
    fpath = os.path.join(main_folder, subfolder,'output',date1)
    os.makedirs(fpath, exist_ok=True)
    
    for file in os.listdir(csv_folder):
        if file.endswith(".csv"):
            csv_file_path = os.path.join(csv_folder, file)
            df = pd.read_csv(csv_file_path)
            csv_data.append(df)
            # shutil.move(csv_file_path,des_path)
            # os.rename(src_path, des_path)

    all_data = pd.concat(csv_data, ignore_index=True)
    csv_filename = f"{os.path.splitext(csv_folder)[0]}"
    print(csv_filename)
    csv = csv_filename.split('\\')
    csv_data.clear()

    if csv[-2] == 'sbi':
        print('i am in sbi')
        final_csv_path = os.path.join(fpath,'Final_sbi.csv',)
        print(fpath,)
        print(all_data)
        print(final_csv_path)
        all_data.to_csv(final_csv_path,index=False)
          

    if csv[-2] == 'birla':
        print('i am in birla')
        final_csv_path = os.path.join(fpath,'Final_birla.csv',)
        print(fpath)
        print(all_data)
        print(final_csv_path)
        all_data.to_csv(final_csv_path,index=False)
    

    if csv[-2] == 'icici':
        print('i am in icici')
        final_csv_path = os.path.join(fpath,'Final_icici.csv',)
        print(fpath)
        print(all_data)
        print(final_csv_path)
        all_data.to_csv(final_csv_path,index=False)
        

D:\Python\ajit\combined_all_code\op1\icici\csv ============
i am in icici
D:\Python\ajit\combined_all_code\op1\icici\output\2024-10-08 9999999999999999999999999
      Folio Number                       Company Name     ISIN Number  \
0    11230632 / 84      western transco power limited   INF109KC11G1    
1    11230632 / 84      western transco power limited   INF109KC11G1    
2    12483571 / 17  chattisgarath wr transmission ltd   INF109KC11G1    
3   15380577 / 05:           adani agri logistics ltd   INF109K01Q49    
4   15380577 / 05:           adani agri logistics ltd   INF109K01Q49    

   Value Date Transaction Type  Amount in INR  NAV in INR  Price in INR  \
0  01/08/2024         Purchase     2899855.01   1318.9677     1318.9677   
1  13/08/2024         Purchase      799960.00   1321.7066     1321.7066   
2  07/08/2024         Purchase     2999850.01   1320.3240     1320.3240   
3  05/08/2024         Purchase    31472426.38    366.5027      366.5027   
4  07/08/2024         Pur

In [ ]:
# import glob
# path =r"D:\Python\ajit\SBI\new_pdf\main\op1"
# all_files = glob.glob(os.path.join(path, "*.csv"))  
# li = []
# for filename in all_files:     
#     print(filename)
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)
# # Modify the transaction_type column based on the condition
# # frame['Transaction Type'] = frame['Transaction Type'].apply(lambda x: 'Purchase' if x.lower().startswith('pu') else 'Redemption')
# frame.to_csv(r"D:\Python\ajit\SBI\new_pdf\main\final.csv",index=False)
# print(frame)

In [ ]:
# ## create Final_csv as per MF Bank

# source = r"D:\Python\ajit\combined_all_code\op1"
# destination = r'D:\Python\ajit\combined_all_code\move_csv'
 
# allfiles = os.listdir(source)
# # allfiles = glob.glob(os.path.join(path, "*.csv"))  
# li = []

# for f in allfiles:
#     src_path = os.path.join(source, f)
#     dst_path = os.path.join(destination, f)
#     df = pd.read_csv(src_path, index_col=None, header=0)
#     li.append(df)
#     os.rename(src_path, dst_path)
    
# com_df = pd.concat(li, axis=0, ignore_index=True)
# # Modify the transaction_type column based on the condition
# com_df['Transaction Type'] = com_df['Transaction Type'].apply(lambda x: 'Purchase' if x.lower().startswith('pu') else 'Redemption')

# csv_filename = f"{os.path.splitext(source)[0]}"
# csv = csv_filename.split('\\')
# if csv[-2] == 'sbi':
#     # print('i am in sbi')
#     com_df.to_csv(r"D:\Python\ajit\SBI\new_pdf\test\csv\sbi\final_csv\final_sbi.csv",index=False)

# if csv[-2] == 'birla':
#     com_df.to_csv(r"D:\Python\ajit\SBI\new_pdf\test\csv\sbi\final_csv\final_birla.csv",index=False)

# if csv[-2] == 'icici':
#     com_df.to_csv(r"D:\Python\ajit\SBI\new_pdf\test\csv\sbi\final_csv\final_icici.csv",index=False)

In [ ]:
# import os
# import pandas as pd

# main_folder = r"D:\Python\ajit\combined_all_code\op1"  
# destination = r'D:\Python\ajit\combined_all_code\move_csv'
# final_ = r'D:\Python\ajit\combined_all_code\final_csv'
# csv_data = []

# for subfolder in ["icici", "sbi", "birla"]:
#     output_folder = os.path.join(main_folder, subfolder, "output")
#     des_path = os.path.join(destination, subfolder)
#     os.makedirs(des_path, exist_ok=True)

#     for file in os.listdir(output_folder):
#         if file.endswith(".csv"):
#             csv_file_path = os.path.join(output_folder, file)
#             print(csv_file_path)
#             df = pd.read_csv(csv_file_path)
#             csv_data.append(df)
#             shutil.move(csv_file_path,des_path)
#             # os.rename(src_path, des_path)

#     all_data = pd.concat(csv_data, ignore_index=True)
#     print(all_data)
#     csv_filename = f"{os.path.splitext(output_folder)[0]}"
#     csv = csv_filename.split('\\')

#     csv_data.clear()
#     if csv[-2] == 'sbi':
#         print('i am in sbi')
#         final_csv_path = os.path.join(final_, 'sbi' +'.csv',)
#         print(final_csv_path,'=========')

#         all_data.to_csv(final_csv_path,index=False)

#     if csv[-2] == 'birla':
#         print('i am in birla')
#         final_csv_path = os.path.join(final_, 'birla' +'.csv',)
#         print(final_csv_path,'=========')
#         all_data.to_csv(final_csv_path,index=False)


#     if csv[-2] == 'icici':
#         print('i am in icici')
#         final_csv_path = os.path.join(final_, 'icici' +'.csv',)
#         print(final_csv_path,'=========')
#         all_data.to_csv(final_csv_path,index=False)
